In [1]:
%cd ../..
%load_ext autoreload
%autoreload 2

/root/capsule/code


In [7]:
import itertools
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import pandas as pd
from pipeline import lab, experiment, ephys, psth_foraging, foraging_analysis

foraging_sessions = (foraging_analysis.SessionTaskProtocol & 'session_task_protocol=100').proj()
import datajoint as dj; dj.conn().connect()

# Add choices and autoregression into linear regression

In [6]:
psth_foraging.UnitPeriodLinearFit.progress()

UnitPeriodLinearFit  Completed 714777 of 714777 (100.0%)   2023-03-29 19:08:31


(0, 714777)

In [15]:
psth_foraging.IndependentVariable()

var_name,desc
choice_ic,ipsi (0) or contra (1)
choice_ic_next,"ipsi (0) or contra (1), next choice"
choice_lr,left (0) or right (1)
contra_action_value,contra action_value
contra_choice_kernel,contra choice_kernel
contra_choice_prob,contra choice_prob
firing_1_back,firing rate of 1 trials back
firing_2_back,firing rate of 2 trials back
firing_3_back,firing rate of 3 trials back
firing_4_back,firing rate of 4 trials back


In [26]:
psth_foraging.LinearModel.load()
psth_foraging.LinearModel()

multi_linear_model,if_intercept Whether intercept is included
"dQ, sumQ, rpe, C*2",1
"dQ, sumQ, rpe, C*2, R*1",1
"dQ, sumQ, rpe, C*2, R*1, t",1
"dQ, sumQ, rpe, C*2, R*5, t",1
"dQ, sumQ, rpe, C*2, t",1
Q_c + Q_i + rpe,1
Q_l + Q_r + rpe,1
Q_rel + Q_tot + rpe,1


## Debug fitting

In [27]:
date, imec, unit = '2021-04-18', 0, 541
unit_key = (ephys.Unit() & (experiment.Session & 'session_date = "2021-04-18"' & 'subject_id = 473361') & {'insertion_number': imec + 1, 'unit_uid': unit}).fetch1("KEY")
key = {**unit_key, 'period': "iti_all", 'multi_linear_model': "dQ, sumQ, rpe, C*2"}    

In [39]:
key = {'subject_id': 473360, 'session': 49, 'insertion_number': 1, 'clustering_method': 'kilosort2', 'unit': 187, 'period': 'delay', 'behavior_model': 'best_aic', 'multi_linear_model': 'dQ, sumQ, rpe, C*2'}

In [41]:
psth_foraging.UnitPeriodLinearFit.populate(key, display_progress=True)

UnitPeriodLinearFit:   0%|          | 0/1 [00:00<?, ?it/s]/opt/conda/lib/python3.9/site-packages/statsmodels/regression/linear_model.py:1752: RuntimeWarning: invalid value encountered in double_scalars
  return 1 - self.ssr/self.centered_tss
/opt/conda/lib/python3.9/site-packages/statsmodels/regression/linear_model.py:1841: RuntimeWarning: invalid value encountered in double_scalars
  return self.mse_model/self.mse_resid
/opt/conda/lib/python3.9/site-packages/statsmodels/regression/linear_model.py:940: RuntimeWarning: divide by zero encountered in log
  llf = -nobs2*np.log(2*np.pi) - nobs2*np.log(ssr / nobs) - nobs2
UnitPeriodLinearFit: 100%|██████████| 1/1 [00:01<00:00,  1.79s/it]


In [36]:
psth_foraging.UnitPeriodLinearFit.progress()

UnitPeriodLinearFit  Completed 4360 of 1906072 (0.2%)   2023-03-29 21:05:17


(1901712, 1906072)

In [31]:
psth_foraging.UnitPeriodLinearFit & 'period = "iti_all"'

subject_id institution 6 digit animal ID,session session number,insertion_number,clustering_method,unit,period,"behavior_model If 'best_aic' etc, the best behavioral model for each session; otherwise --> Model.model_id",multi_linear_model,actual_behavior_model,model_r2 r square,model_r2_adj r square adj.,model_p,model_bic,model_aic
473361,47,1,kilosort2,541,iti_all,best_aic,"dQ, sumQ, rpe, C*2",16,0.579215,0.574134,0.0,2892.07,2867.83
473361,47,1,kilosort2,541,iti_all,best_aic,"dQ, sumQ, rpe, C*2, R*1",16,0.63542,0.630111,0.0,2831.58,2803.32
473361,47,1,kilosort2,541,iti_all,best_aic,"dQ, sumQ, rpe, C*2, R*1, t",16,0.66198,0.656223,0.0,2805.93,2773.63
473361,47,1,kilosort2,541,iti_all,best_aic,"dQ, sumQ, rpe, C*2, R*5, t",16,0.662665,0.653457,0.0,2804.22,2755.88
473361,47,1,kilosort2,541,iti_all,best_aic,"dQ, sumQ, rpe, C*2, t",16,0.634914,0.62961,0.0,2838.48,2810.19
473361,47,1,kilosort2,541,iti_all,best_aic,Q_c + Q_i + rpe,16,0.395207,0.390856,2.8026e-45,3038.52,3022.35
473361,47,1,kilosort2,541,iti_all,best_aic,Q_l + Q_r + rpe,16,0.395207,0.390856,2.8026e-45,3038.52,3022.35
473361,47,1,kilosort2,541,iti_all,best_aic,Q_rel + Q_tot + rpe,16,0.395207,0.390856,2.8026e-45,3038.52,3022.35


In [38]:
psth_foraging.schema.jobs.fetch('key', limit=1)

array([{'subject_id': 473360, 'session': 49, 'insertion_number': 1, 'clustering_method': 'kilosort2', 'unit': 187, 'period': 'delay', 'behavior_model': 'best_aic', 'multi_linear_model': 'dQ, sumQ, rpe, C*2'}],
      dtype=object)